In [ ]:
!pip install ripgrepy
!apt install ripgrep 

In [ ]:
# tmp_df.to_csv('your.csv', index=False) # don't write unnamed: 0 index col
# tmp_df.read_csv('your.csv', index_col=0) #or if did, tell pandas to use 1st col as index

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from pathlib import Path
filepath = Path('nogit_data', 'list_of_post_contents.csv')
#df = pd.read_csv(filepath, nrows=10_00)
df = pd.read_csv(filepath).dropna(subset='time_downloaded')
#df

In [ ]:
df.post_ordinal.apply(type).unique()

In [ ]:
#unique_types = df.post_ordinal.apply(type).unique()

In [ ]:
def comma_to_int(df, col_name):
    df[col_name] = df[col_name].apply(lambda x: x.replace(',', '') if type(x) == str else x) 
    df[col_name] = df[col_name].astype(float)
    df[col_name] = df[col_name].astype(int)
    return df

In [ ]:
df.post_id.value_counts()

In [ ]:
# post-id is unique -- we can use it to drop duplicate downloads of posts

filepath = Path('nogit_data', 'list_of_post_contents.csv')
#df = pd.read_csv(filepath, nrows=10_00)
df = pd.read_csv(filepath)
df = df.dropna(subset='time_downloaded')
df = df.drop_duplicates('post_id')
df.to_csv('nogit_data/list_of_post_contents_nona.csv', index=False)

In [ ]:
# how to covnert join date data to datetime pandas
# how to time runs

In [ ]:
df[['join_date_data', 'join_date_readable']]

In [ ]:
%%time 
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
#
df.join_date_data.parallel_apply(lambda x: pd.to_datetime(x, unit = 's'))

In [ ]:
%%time
df.join_date_readable.parallel_apply(lambda x: pd.to_datetime(x))

In [ ]:
# Convert column values like "2.0" to "2" and "2,000" to "2000" (as integers)
df = pd.read_csv('nogit_data/list_of_post_contents_nona.csv')
for column in ['post_ordinal', 'author_num_posts', 'thread_page_num', 'thread_max_pages', 'num_quotes']:
    print(column)
    df = comma_to_int(df, column)

df.to_csv('nogit_data/list_of_post_contents_nona_nocomma.csv', index=False)


In [ ]:
df = pd.read_csv('nogit_data/list_of_post_contents_nona_nocomma.csv')
df.drop(columns=['comment', 'join_date_data', 'posted_date_data'], inplace=True)
df.to_csv('nogit_data/list_of_post_contents_nona_nocomma_nodate.csv', index=False)
df.drop(columns=['num_likers'], inplace=True)
df.to_csv('nogit_data/list_of_post_contents_nona_nocomma_nodate_nonumlikers.csv', index=False)

In [ ]:
pd.read_csv('nogit_data/list_of_post_contents_nona_nocomma_nodate_nonumlikers.csv')

In [ ]:
readable = ['author', 'posted_date_readable', 'src_category_name', 'thread_page_url', 'post_ordinal', 'post_text']

df[readable].to_csv('nogit_data/post_data_only.csv', index=False)

In [ ]:
#df[df.post_id=='post-874660'].thread_page_url.to_list()

In [ ]:
with pd.option_context('display.max_rows', 100, 'display.max_columns', 30):
    display(df[df.duplicated(subset=['post_ordinal', 'post_id', 'post_text'])])

In [ ]:
# df.dropna(subset=['post_text'])#, inplace=True) --> mostly, having only quoted text as a post
print(f'{df.shape=}')
df.drop_duplicates(subset=['author', 'post_text'])#, inplace=True)
print(f'{df.shape=}')
print(f'{df.shape=}')


In [ ]:
#df.drop_duplicates().info()
#df.groupby(df.columns.tolist(),as_index=False).first().info()

In [ ]:
#df[df.post_text == null ]

In [ ]:
df.dtypes

In [ ]:
for col in df.columns:
    unique_types = df[col].apply(type).unique()
    if len(unique_types) > 1:
        print(col, unique_types)

# fix string columns

In [ ]:
df.dtypes

In [ ]:
df[df.post_text.isna()].iloc[0].post_text 

In [ ]:
df['post_text'] = df['post_text'].fillna('')#, inplace=True)

In [ ]:
df[df.post_text.isna()]

In [ ]:
df[df.post_text=='']

In [ ]:
df[readable].to_csv('nogit_data/post_data_only.csv', index=False)

In [ ]:
from ripgrepy import Ripgrepy

In [ ]:
rg = Ripgrepy('undercover', str(Path('nogit_data', 'post_data_only.csv')))
rg.before_context(2).pretty().run()#.as_string
rg = Ripgrepy('undercover', str(Path('nogit_data', 'post_data_only.csv')))
rg.count_matches().pretty().run()#.as_string

In [ ]:
!rg "undercover" nogit_data/post_data_only.csv --count-matches


In [ ]:
rg = Ripgrepy('undercover', str(Path('nogit_data', 'post_data_only.csv')))
rg.stats().pretty().run()

In [ ]:
#Date downloaded - author - author url - post 
readable = ['time_downloaded', 'posted_date_readable', 'src_category_name','thread_page_url', 'author', 'author_url', 'post_text' ]
df[readable].to_csv('nogit_data/hackathon_ripgrep_post_contents.csv')


In [ ]:
df.columns

In [ ]:
posts = pd.read_csv('nogit_data/hackathon_ripgrep_post_contents.csv')
posts

In [ ]:
import re
import base64
from pandas.util import hash_pandas_object

print(df.columns)
all_authors = pd.Series(df.author.unique())
    # author_re = re.compile('|'.join(re.escape(a) for a in all_authors))

    author_ids = {a: base64.b64encode(a.to_bytes(8, 'big')) for a in hash_pandas_object(all_authors)}
    print(author_ids)